# Loading Dataset From kaggle

In [ ]:
!pip install kaggle

In [ ]:
from google.colab import files

# Upload the kaggle.json file
files.upload()

# Move kaggle.json to the .kaggle directory
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/

In [ ]:
!chmod 600 ~/.kaggle/kaggle.json

In [ ]:
!kaggle datasets download -d chrisfilo/urbansound8k

# import libraries

In [ ]:
import os
import math, random

import torch
import torch.nn as nn
import torch.optim as optim
from torch.nn import init
from torchaudio import transforms
from torch.utils.data import random_split
from torch.utils.data import DataLoader, Dataset, random_split
import torch.nn.functional as F
import torchaudio

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from PIL import Image
from tqdm import tqdm